## Using time series for predictive mainteneance of turbofan engines

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Data import
data_dir = "../CMAPSSData/"

In [3]:
!ls ../CMAPSSData/

ls: cannot access '../CMAPSSData/': No such file or directory


In [4]:
sensor_colnames = [f"sensor{i}" for i in range(1,22, 1)]

In [5]:
engine_colnames = ["unit_number", "cycles", "operational_setting_1", "operational_setting_2", "operational_setting_3"] + sensor_colnames

In [6]:
train_fd001_raw = pd.read_csv(f"{data_dir}train_FD001.txt", delim_whitespace=True, names=engine_colnames)
train_fd002_raw = pd.read_csv(f"{data_dir}train_FD002.txt", delim_whitespace=True, names=engine_colnames)
train_fd003_raw = pd.read_csv(f"{data_dir}train_FD003.txt", delim_whitespace=True, names=engine_colnames)
train_fd004_raw = pd.read_csv(f"{data_dir}train_FD004.txt", delim_whitespace=True, names=engine_colnames)

FileNotFoundError: [Errno 2] No such file or directory: '../CMAPSSData/train_FD001.txt'

In [ ]:
test_fd001_raw = pd.read_csv(f"{data_dir}test_FD001.txt", delim_whitespace=True, names=engine_colnames)
test_fd002_raw = pd.read_csv(f"{data_dir}test_FD002.txt", delim_whitespace=True, names=engine_colnames)
test_fd003_raw = pd.read_csv(f"{data_dir}test_FD003.txt", delim_whitespace=True, names=engine_colnames)
test_fd004_raw = pd.read_csv(f"{data_dir}test_FD004.txt", delim_whitespace=True, names=engine_colnames)

In [ ]:
rul_fd001_raw = pd.read_csv(f"{data_dir}RUL_FD001.txt", names=["rul_fd001"], squeeze=True)
rul_fd002_raw = pd.read_csv(f"{data_dir}RUL_FD002.txt", names=["rul_fd002"], squeeze=True)
rul_fd003_raw = pd.read_csv(f"{data_dir}RUL_FD003.txt", names=["rul_fd003"], squeeze=True)
rul_fd004_raw = pd.read_csv(f"{data_dir}RUL_FD004.txt", names=["rul_fd004"], squeeze=True)

In [ ]:
#rul_df = pd.DataFrame({"rul_fd001": rul_fd001_raw, "rul_fd002": rul_fd002_raw, "rul_fd003": rul_fd003_raw, "rul_fd004": rul_fd004_raw}, dtype=np.int)

In [ ]:
#rul_fd001_raw

### Assumptions

- data in rul_ are useful life of a particular unit
- unit number and trejectories are synonyms
- data in rul_ are ordered by thier unit number
- rul_df represent test set remaining useful life

In [ ]:
rul_df["unit_number"] = list(range(1, rul_df.shape[0] + 1))

In [ ]:
cols = rul_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
rul_df = rul_df[cols]

In [ ]:
rul_df = rul_df.set_index("unit_number")

In [ ]:
rul_df.head(2)

### Model data transformation

In [ ]:
train_fd001 = train_fd001_raw.copy()
train_fd002 = train_fd002_raw.copy()
train_fd003 = train_fd003_raw.copy()
train_fd004 = train_fd004_raw.copy()

In [ ]:
test_fd001 = test_fd001_raw.copy()
test_fd002 = test_fd002_raw.copy()
test_fd003 = test_fd003_raw.copy()
test_fd004 = test_fd004_raw.copy()

In [ ]:
# assuming linear reduction on remaining useful life
train_fd001["rul"] = train_fd001.groupby(["unit_number"], group_keys=False).apply(lambda g: max(g.cycles) - g.cycles)
train_fd002["rul"] = train_fd002.groupby(["unit_number"], group_keys=False).apply(lambda g: max(g.cycles) - g.cycles)
train_fd003["rul"] = train_fd003.groupby(["unit_number"], group_keys=False).apply(lambda g: max(g.cycles) - g.cycles)
train_fd004["rul"] = train_fd004.groupby(["unit_number"], group_keys=False).apply(lambda g: max(g.cycles) - g.cycles)

In [ ]:
train_fd001["train_data"] = "fd001"
train_fd002["train_data"] = "fd002"
train_fd003["train_data"] = "fd003"
train_fd004["train_data"] = "fd004"

In [ ]:
test_fd001["test_data"] = "fd001"
test_fd002["test_data"] = "fd002"
test_fd003["test_data"] = "fd003"
test_fd004["test_data"] = "fd004"

In [ ]:
train = pd.concat([train_fd001, train_fd002, train_fd003, train_fd004], axis=0)

In [ ]:
train = train.set_index([ "train_data", "unit_number", "cycles"])

In [ ]:
train.head(2)

In [ ]:
train.tail(2)

In [ ]:
train.index

In [ ]:
test = pd.concat([test_fd001, test_fd002, test_fd003, test_fd004], axis=0)

In [ ]:
test = test.set_index([ "test_data", "unit_number", "cycles"])

In [ ]:
test.head(2)

In [ ]:
test.tail(2)

In [ ]:
# How to index into the training set
# Select the first unit of fd001 dataset # Can unstack with tarain.unstack()
train.loc[("fd001", 1)]

In [ ]:
train.head(2)

### Using clipped RUL
Assuming cliiped RUL over one that decreases linearly overtime may better reflect real operating condtions. See 

https://towardsdatascience.com/the-importance-of-problem-framing-for-supervised-predictive-maintenance-solutions-cc8646826093

In [ ]:
train.loc[("fd001", 1)].rul.clip(upper=125)

In [ ]:
train["rul_clipped"] = train.rul.clip(upper=125)

In [ ]:
train["rul_clipped"]

### Model using time series

In [ ]:
train.index

In [ ]:
train.loc["fd001"]#["unit_number"]

In [ ]:
train.loc[("fd001", 1), :]

In [ ]:
train.loc[("fd001", slice(None)), :]

In [ ]:
###train.loc[("fd001", slice(None)), :] ###train.loc[("fd001"), :]  ### train.loc[("fd001")]

In [ ]:
train.loc[("fd001")].index.get_level_values("unit_number").unique() #train.index.get_level_values("unit_number").unique()

In [ ]:
train.head(2)

In [ ]:
def plot_feature(train_data, unit, feature):
    #plt.figure(figsize=(12, 6))
    #plt.plot("rul", "sensor2", data=train.loc[("fd001", 1)])
    #plt.plot("rul", feature, data=train.loc[(train_data, unit)])
    plt.plot(feature, data=train.loc[(train_data, unit)])
    #plt.xlim(0, 250)
    plt.xlabel("cycles")
    plt.ylabel(feature)

In [ ]:
#plt.plot("sensor2", data=train.loc[("fd001", 1)])

In [ ]:
#def plot_sensor(sensor):
#    plt.figure(figsize = (12, 6))
#    
#    for unit in train.loc[("fd001", )]
#    

In [ ]:
#for unit in train.loc[("fd001")].index.get_level_values("unit_number").unique():
#    print(unit)

In [ ]:
plot_feature("fd001", 1, "sensor2")

In [ ]:
print(train.loc[("fd001")].index.get_level_values("unit_number").unique())

In [ ]:
drop_sensors = ['sensor1','sensor5','sensor6','sensor10','sensor16','sensor18','sensor19']
drop_settings =  ["operational_setting_1", "operational_setting_2", "operational_setting_3"]
drop_targets = ["rul"] # ["rul", rul_clipped"]
drop_labels = drop_sensors + drop_settings + drop_targets
drop_test_labels = drop_sensors + drop_settings
print(drop_labels)

In [ ]:
Xtrain = train.loc[("fd001")].drop(drop_labels, axis=1)

In [ ]:
Xtrain.head(2)

In [ ]:
remaining_sensors = list(Xtrain.columns.difference(["rul", "rul_clipped"]))
print(remaining_sensors)

In [ ]:
lag1 = [col + '_lag_1' for col in remaining_sensors]
print(lag1)

In [ ]:
Xtrain[lag1] = Xtrain[remaining_sensors].shift(1)

In [ ]:
Xtrain.head(4)

In [ ]:
Xtrain.dropna(inplace=True)
ytrain = Xtrain.pop('rul_clipped')

In [ ]:
Xtrain.info()

In [ ]:
ytrain

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(Xtrain, ytrain)

In [ ]:
lm

In [ ]:
test

In [ ]:
rul_fd001_raw

In [ ]:
def evaluate(y_true, y_hat, label='test'):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    variance = r2_score(y_true, y_hat)
    print('{} set RMSE:{}, R2:{}'.format(label, rmse, variance))

In [ ]:
Xtest = test.loc[("fd001")].drop(drop_test_labels, axis=1)

In [ ]:
Xtest[lag1] = Xtest[remaining_sensors].shift(1)

In [ ]:
Xtest.dropna(inplace=True)

In [ ]:
Xtest

In [ ]:
#Xtrain

In [ ]:
y_hat_test = lm.predict(Xtest)

In [ ]:
print(y_hat_test)

In [ ]:
len(y_hat_test)